In [1]:
from typing import Callable, Iterator, List, Union
from osgeo import gdal, gdal_array
import numpy as np
import SharedArray as sa
import skmap_bindings


data = np.zeros((12,5), dtype=np.float32)

data[[8, 6, 3, 2], 0:2] = 200.
data[[0, 1, 3, 5], 0:3] = 200.
data[[0, 1], 4:5] = 200.
indices_matrix = [[0,1,2,3,4],
                  [5,6,7,8,9]]
result_indices = [10, 11]
th = 169.
n_threads = 96
result_scaling = 100.
skmap_bindings.computeBsf(data, n_threads, indices_matrix, result_indices, th, result_scaling)
print(data)

[[200. 200. 200.   0. 200.]
 [200. 200. 200.   0. 200.]
 [200. 200.   0.   0.   0.]
 [200. 200. 200.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [200. 200. 200.   0.   0.]
 [200. 200.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [200. 200.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [ 20.  20.  40. 100.  60.]
 [ 40.  40.  80. 100. 100.]]


In [3]:
from typing import Callable, Iterator, List, Union
from osgeo import gdal, gdal_array
import numpy as np
import SharedArray as sa
import skmap_bindings

data = np.zeros((12,4), dtype=np.float32)
reordered_data = np.zeros((5,12), dtype=np.float32)
data[[8, 6, 3, 2], 0:2] = 200.
data[[0, 1, 3, 5], 0:3] = 99.
data[[0, 1], 4:5] = 1.
indices_matrix = [[0,10,2],
                  [5,6,7],
                  [8,0,2],
                  [1,4,5],
                  [10,7,5]]
n_threads = 96
skmap_bindings.reorderArray(data, n_threads, reordered_data, indices_matrix)
print("data")
print(data)
print("reordered_data")
print(reordered_data)

inverse_reordered_data = np.zeros((15,4), dtype=np.float32)
subrows = np.arange(0, 3)
rows = np.arange(0, 5)
subrows_grid, rows_grid = np.meshgrid(subrows, rows)
inverse_idx = np.empty((inverse_reordered_data.shape[0],2), dtype=np.uintc)
inverse_idx[:,0] = rows_grid.flatten()
inverse_idx[:,1] = subrows_grid.flatten()
print("inverse_idx")
print(inverse_idx)
skmap_bindings.inverseReorderArray(data, n_threads, inverse_reordered_data, inverse_idx)
print("inverse_reordered_data")
print(inverse_reordered_data)

data
[[ 99.  99.  99.   0.]
 [ 99.  99.  99.   0.]
 [200. 200.   0.   0.]
 [ 99.  99.  99.   0.]
 [  0.   0.   0.   0.]
 [ 99.  99.  99.   0.]
 [200. 200.   0.   0.]
 [  0.   0.   0.   0.]
 [200. 200.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]]
reordered_data
[[ 99.  99.  99.   0.   0.   0.   0.   0. 200. 200.   0.   0.]
 [ 99.  99.  99.   0. 200. 200.   0.   0.   0.   0.   0.   0.]
 [200. 200.   0.   0.  99.  99.  99.   0. 200. 200.   0.   0.]
 [ 99.  99.  99.   0.   0.   0.   0.   0.  99.  99.  99.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.  99.  99.  99.   0.]]
inverse_idx
[[0 0]
 [0 1]
 [0 2]
 [1 0]
 [1 1]
 [1 2]
 [2 0]
 [2 1]
 [2 2]
 [3 0]
 [3 1]
 [3 2]
 [4 0]
 [4 1]
 [4 2]]
inverse_reordered_data
[[ 99.  99.  99.   0.]
 [ 99.  99.  99.   0.]
 [200. 200.   0.   0.]
 [ 99.  99.  99.   0.]
 [200. 200.   0.   0.]
 [ 99.  99.  99.   0.]
 [200. 200.   0.   0.]
 [ 99.  99.  99.   0.]
 [  0.   0.   0.   0.]
 [ 99.  99.  99.   0.]
 [  0.   0.   0.

In [1]:
import numpy as np
import skmap_bindings
from skmap.misc import find_files
import time
import tempfile
from pathlib import Path
from osgeo import gdal, gdal_array
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

x_off = 2
y_off = 2
x_size = 4000
y_size = 4000
n_pix = x_size * y_size

tile = '120E_46N'
file_loc = f'http://192.168.1.30:8333/gpw/landmask/{tile}.tif'
conf_GDAL = {"CPL_VSIL_CURL_ALLOWED_EXTENSIONS": ".tif",
             "GDAL_HTTP_VERSION": "1.0"}
mask = np.zeros((2,n_pix), dtype=np.float32)
n_threads = 96
skmap_bindings.readData(mask, n_threads, [file_loc,], [0,], x_off, y_off, x_size, y_size, [1,], conf_GDAL)
print(np.sum(mask))
print(mask.size)
mask[1,:] = mask[0,:] * 100.
mask[0,:] = mask[0,:] * 200.
no_data_value = 255
compression_command = "gdal_translate -co COMPRESS=deflate -co ZLEVEL=9 -co TILED=TRUE -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024"
write_data_indices = [1, 0]
s3_paths = ["gaia/tmp-eumap-ai4sh/test/test/class",
             "gaia/tmp-eumap-ai4sh/test/test"]
skmap_bindings.writeByteData(mask, n_threads, conf_GDAL, file_loc, './logo', ['test2','test1'], write_data_indices,
        x_off, y_off, x_size, y_size, no_data_value, compression_command, s3_paths)



13992422.0
32000000


[[0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]]
[[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]]
